
## Load Data

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [175]:
dataset=pd.read_csv('data.csv')


In [176]:
data=pd.read_csv('data.csv')


In [177]:
dataset.head()


In [178]:
for col in data:
    print (type(data[col][1]))

In [179]:
data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')


In [180]:
data['timestamp'] = pd.to_datetime(data['timestamp'], format = '%d/%m/%Y %H:%M:%S')


In [181]:
data['timestamp']

In [182]:
# DATE TIME STAMP FUNCTION
column_1 = data.iloc[:, 0]

db = pd.DataFrame({"year": column_1.dt.year,
              "month": column_1.dt.month,
              "day": column_1.dt.day,
              "hour": column_1.dt.hour,
              "dayofyear": column_1.dt.dayofyear,
              "week": column_1.dt.week,
              "weekofyear": column_1.dt.weekofyear,
              "dayofweek": column_1.dt.dayofweek,
              "weekday": column_1.dt.weekday,
              "quarter": column_1.dt.quarter,
             })


In [183]:
dataset1=dataset.drop('timestamp',axis=1)

In [184]:
data1=pd.concat([db,dataset1],axis=1)

## Data Analysis

In [185]:
data1.info()

In [186]:
data1.dropna(inplace=True)

In [187]:
data1.head()

## Data Visualization & Analysis

In [188]:
sns.pairplot(data1,hue='act363')

In [189]:
sns.boxplot(x='act379' ,y='hour' ,data=data1, palette='winter_r')

In [190]:
sns.boxplot(x='act13' ,y='hour' ,data=data1 , palette='winter_r')

In [191]:
sns.boxplot(x='act323' ,y='hour' ,data=data1, palette='winter_r')

In [192]:
sns.boxplot(x='act363' ,y='hour' ,data=data1, palette='winter_r')

In [193]:
df = pd.DataFrame(data=data1, columns=['act13', 'hour', 'day'])

In [194]:
df.plot.hexbin(x='act13',y='hour',gridsize=25)

In [195]:
df.plot(legend=False)

In [196]:
df1 = pd.DataFrame(data=data1, columns=['act13', 'act323', 'act379'])

In [197]:
df1.plot.kde()

## X & Y array

In [198]:
X=data1.iloc[:,[1,2,3,4,6,16,17]].values


In [199]:
X

In [200]:
y=data1.iloc[:,[10,11,12,13,14,15]].values

In [201]:
y

In [202]:
## Splitting the data

In [203]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

## Creating & Training KNN Model

In [204]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)

In [205]:
knn.score(X_test,y_test)

In [206]:
knn.score(X_train,y_train)

### Elbow Method For optimum value of K

In [207]:
error_rate = []
for i in range(1,140):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [208]:
plt.figure(figsize=(10,6))
plt.plot(range(1,140),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=5)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

## Creating & Training Decision Tree Model

In [209]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(max_depth=500, random_state=300)

In [210]:
dtree.fit(X_train,y_train)

In [211]:
y_pred=dtree.predict(X_test)

In [212]:
dtree.score(X_test,y_test)

In [213]:
dtree.score(X_train,y_train)

In [214]:
y_pred

In [215]:
treefeatures=dtree.feature_importances_
indices = np.argsort(treefeatures)


In [216]:
treefeatures

In [217]:
features = data1.iloc[:,[1,2,3,4,6,16,17]]

In [218]:
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), treefeatures[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')

from joblib import dump
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
# Save the trained model
dump(rfc, 'crime_prediction_model.joblib')

y_pred=rfc.predict(X_test)
rfc.score(X_test,y_test)
rfc.score(X_train,y_train)
om=rfc.feature_importances_
indices = np.argsort(om)

om
features = data1.columns
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), om[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')

## Tree Visualization

In [ ]:

feature_names=[ 'dayofweek', 'dayofyear', 'hour', 'month', 'week','latitude', 'longitude']
from io import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydot
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
dot_data = StringIO()
export_graphviz(dtree, out_file=dot_data,feature_names=feature_names,filled=True)

graph = pydot.graph_from_dot_data(dot_data.getvalue())
Image(graph[0].create_png())

## Creating & Training Random Tree Model

In [ ]:
from joblib import dump
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
# Save the trained model
dump(rfc, 'crime_prediction_model.joblib')

y_pred=rfc.predict(X_test)
rfc.score(X_test,y_test)
rfc.score(X_train,y_train)
om=rfc.feature_importances_
indices = np.argsort(om)

om
features = data1.columns
plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), om[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')

In [ ]:
!ipython nbconvert --to slides CrimePrediction.ipynb


In [ ]:
!ipython nbconvert --to pdf CrimePrediction.ipynb